<h2><font color="#004D7F" size=6>Módulo 7. Proyectos de Machine Learning</font></h2>



<h1><font color="#004D7F" size=5>2. Proyecto de clasificación binaria</font></h1>

<br><br>
<div style="text-align: right">
<font color="#004D7F" size=3>Manuel Castillo Cara</font><br>
<font color="#004D7F" size=3>Machine Learning con Python</font><br>

---

<h2><font color="#004D7F" size=5>Índice</font></h2>
<a id="indice"></a>

* [1. Introducción](#section1)
    * [1.1. Importar librerías](#section11)
    * [2.1. Cargar el dataset](#section12)
* [2. Estadística descriptiva](#section2)
    * [2.1. Dimensiones del dataset](#section21)
    * [2.2. Visualización de datos](#section22)
    * [2.3. Resumen estadístico](#section23)
    * [2.4. Distribución de clase](#section24)
    * [2.5. Tipo de datos](#section25)
* [3. Visualización del dataset](#section3)
    * [3.1. Gráficos Univariado](#section31)
    * [3.2. Gráficos multivariados](#section32)
* [4. Fase de modelado](#section4)
    * [4.1. Crear conjunto de validación](#section41)
    * [4.2. Evaluación de línea base](#section42)
    * [4.3. Evaluar modelos: estandarización](#section43)
* [5. Fase de Optimización](#section5)
    * [5.1. Optimización de k-NN](#section51)
    * [5.2. Optimización de SVM](#section52)
* [6. Agoritmos ensamblados](#section6)
* [7. Fase de forecasting](#section7)

In [ ]:
# Permite ajustar la anchura de la parte útil de la libreta (reduce los márgenes)
from IPython.core.display import display, HTML
display(HTML("<style>.container{ width:98% }</style>"))

---

<a id="section1"></a>
# <font color="#004D7F"> 1. Introducción</font>


El enfoque de este proyecto será el conjunto de datos [Sonar Mines vs Rocks](https://archive.ics.uci.edu/ml/datasets/Connectionist+Bench+(Sonar,+Mines+vs.+Rocks)). El problema es predecir objetos de metal o roca a partir de los datos de retorno de la sonda. Cada patrón es un conjunto de 60 números en el rango de 0.0 a 1.0. Cada número representa la energía dentro de una banda de frecuencia particular, integrada durante un cierto período de tiempo. La etiqueta asociada con cada registro contiene la letra "R" si el objeto es una roca y "M" si es una mina (cilindro de metal). Los números en las etiquetas están en orden creciente de ángulo de aspecto, pero no codifican el ángulo directamente.


<a id="section11"></a>
## <font color="#004D7F"> 1.1. Importar librerías</font>

Primero, importemos todos los módulos, funciones y objetos que vamos a utilizar en este tutorial.

In [1]:
# Classification Project: Sonar rocks or mines

# Load libraries
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Load sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

import warnings
warnings.filterwarnings('ignore')

<a id="section12"></a>
## <font color="#004D7F"> 1.2. Cargar el dataset</font>

El conjunto de datos de Sonar se puede descargar desde el repositorio de UCI Machine Learning.

In [ ]:
# Load dataset
filename = 'data/sonar.all-data.csv'
dataset = pd.read_csv(filename, header=None)
dataset

<div style="text-align: right"> <font size=5>
    <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a>
</font></div>

---

<a id="section2"></a>
# <font color="#004D7F"> 2. Estadística descriptiva</font>

Ahora es el momento de estudiar nuestros datos. En este paso vamos a echar un vistazo a los datos de varias maneras diferentes:
* Dimensiones del conjunto de datos.
* Visualización de datos.
* Resumen estadístico de todos los atributos.
* Desglosar las instancias en cada clase.
* Tipo de atributos

<a id="section21"></a>
## <font color="#004D7F"> 2.1. Dimensiones del conjunto de datos</font>

Podemos tener una idea rápida de cuántas instancias (filas) y cuántos atributos (columnas) hay en nuestro conjunto de datos con la propiedad `shape`. 

In [ ]:
# shape
???

Puede verse 208 instancias y 61 atributos (incluyendo la clase).

<a id="section22"></a>
## <font color="#004D7F"> 2.2. Visualización de datos</font>

También siempre es una buena idea ver como están representados los datos con la función `head()`

In [ ]:
# head
pd.set_option('display.width', 100)
???

Podemos ver las primeras 5 filas de los datos y todas las columnas con la misma distribución.

<a id="section23"></a>
## <font color="#004D7F"> 2.3. Resumen estadístico</font>

En este caso podemos echar un vistazo a un resumen de cada atributo con la función `describe()`. Esto incluye la media, los valores mínimo y máximo, así como algunos percentiles.

In [ ]:
# descriptions
pd.set_option('display.precision', 3)
???

<a id="section24"></a>
## <font color="#004D7F"> 2.4. Distribución de clase</font>

Finalmente, veamos ahora la cantidad de instancias (filas) que pertenecen a cada clase. Podemos ver esto como una cuenta absoluta y como un porcentaje con la función `groupby('class').size()`

In [ ]:
# class distribution
???

Podemos ver que cada clase se encuentra más o menos balanceada entre las dos clases.

<a id="section25"></a>
## <font color="#004D7F"> 2.5. Tipo de datos</font>

Finalmente, veamos el tipo de variable de cada atributo con la propiedad `dtypes`.

In [ ]:
# types
???

Podemos ver que todos los atributos son numéricos (flotantes) y que el valor de la clase se ha leído como un objeto.

<a id="section311"></a>
### <font color="#004D7F"> Valores NaN</font>
Es importante también verificar que no tenemos valores NaN en nuestro dataset. En nuestro caso podemos observar que no tenemos ningún valor NaN en las características.

In [ ]:
#Comprobamos si existen NaN
print(dataset.isna().sum().sum())

<a id="section311"></a>
### <font color="#004D7F">Características y Target</font>
Por último, tenemos que separa las variables características del target de manera que tendremos dos dataframe

In [14]:
# Seleccionamos las características
sonar_data = dataset.drop(dataset.columns[60], axis=1)
# Seleccionamos el target
sonar_target = dataset.iloc[:, 60]

In [ ]:
sonar_data

In [ ]:
sonar_target


<div style="text-align: right"> <font size=5>
    <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a>
</font></div>

---

<a id="section3"></a>
# <font color="#004D7F"> 3. Visualización del conjunto de datos</font>

Ahora tenemos una idea básica sobre los datos. Necesitamos extender eso con algunas visualizaciones. Vamos a ver dos tipos de gráficos de visualización de datos:
* Gráficos univariados para comprender mejor cada atributo.
* Gráficos multivariados para comprender mejor las relaciones entre los atributos.

<a id="section31"></a>
## <font color="#004D7F"> 3.1. Gráficos Univariados</font>

Comenzamos con algunas gráficas univariadas, es decir, gráficas de cada variable individual. Es útil con la visualización tener una forma de referirse solo a los atributos de entrada y, por otro lado, solo a los atributos de salida. 

<a id="section311"></a>
### <font color="#004D7F"> Boxplots</font>
Dado que las variables de entrada son numéricas, podemos crear Gráficas Boxplots de cada uno. Podemos ver que los atributos tienen diferenciales bastante diferentes. Dado que las escalas son las mismas, puede sugerir algún beneficio al estandarizar los datos para el modelado para alinear todos los medios.

In [ ]:
# box and whisker plots - Matplotlib
fig = plt.figure(figsize = (20,20))
ax = fig.gca()
(dataset.plot(ax=ax, kind= 'box', subplots=True, layout=(9,7))
)
plt.show()

<div class="alert alert-block alert-warning">
<i class="fa fa-exclamation-circle" aria-hidden="true"></i>

Importante: Hay un bug en Matplotlib y es por eso que hay que poner el `rename`. [Referencia 1](https://stackoverflow.com/questions/53385592/seaborn-issue-with-catplot) y [Referencia 2](https://stackoverflow.com/questions/59199393/i-get-an-error-when-i-use-boxplot-from-pandas-with-subplots)

Si no sale sin `rename` utiliza este código:
```python
    (dataset.rename(columns=lambda x: str(x))
        .plot(ax=ax, kind= 'box', subplots=True, layout=(9,7))
    )  
```
</div>

<a id="section311"></a>
### <font color="#004D7F"> Histograma</font>
También podemos crear un histograma de cada variable de entrada para tener una idea de la distribución. Parece que quizás dos de las variables de entrada tienen una distribución gaussiana. Es útil tener en cuenta que podemos usar algoritmos que pueden explotar esta suposición. 

In [ ]:
# histograms - Matplotlib
fig = plt.figure(figsize = (20,20))
ax = fig.gca()
dataset.hist(ax=ax)
plt.show()

<a id="section311"></a>
### <font color="#004D7F"> Diagrama de densidad</font>
Esto es útil, puede ver que muchos de los atributos tienen una distribución sesgada. Una transformación de potencia como una transformación de Box-Cox que puede corregir el sesgo en las distribuciones podría ser útil.

In [ ]:
# density
fig = plt.figure(figsize = (20,20))
ax = fig.gca()
dataset.plot(ax=ax, kind='density', subplots=True, layout=(8,8), sharex=False, legend=False)
plt.show()

In [ ]:
dataset.plot.density(figsize=(6, 60), subplots=True, yticks=[])
pass

<a id="section32"></a>
## <font color="#004D7F"> 3.2. Gráficos multivariados</font>

Ahora podemos ver las interacciones entre las variables. 

<a id="section321"></a>
### <font color="#004D7F"> Correlación entre atributos</font>
Parece que también hay alguna estructura en el orden de los atributos. El amarilo alrededor de la diagonal sugiere que los atributos que están uno al lado del otro generalmente están más correlacionados entre sí. Los parches azules también sugieren una correlación negativa moderada, los atributos adicionales están lejos unos de otros en el orden. Esto tiene sentido si el orden de los atributos se refiere al ángulo de los sensores para el sonido del sonar.

In [ ]:
# correlation matrix
fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(111)
cax = ax.matshow(sonar_data.corr(), vmin=-1, vmax=1, interpolation='none')
fig.colorbar(cax)
plt.show()

In [ ]:
correlation = sonar_data.corr()
plt.figure(figsize=(50,50))
ax = sns.heatmap(correlation, vmax=1, square=True, annot = True, cmap = 'viridis')
# Esto se ponde debido al bug de Matplotlib 3.1.1 (quitarlo en versiones diferentes)
#bottom, top = ax.get_ylim()
#ax.set_ylim(bottom + 0.5, top - 0.5)
# ----------
plt.title('Correlación entre variables')
plt.show()

<div style="text-align: right"> <font size=5>
    <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a>
</font></div>

---

<a id="section4"></a>
# <font color="#004D7F"> 4. Fase de modelado</font>

Ahora es el momento de crear algunos modelos de datos y estimar su Accruacy en datos no etiquetados. Esto es lo que vamos a cubrir en este paso:
* Crear conjunto de validación.
* Evaluación de línea base.
* Evaluar algoritmos: Estandarización.

<a id="section41"></a>
## <font color="#004D7F"> 4.1. Crear conjunto de validación</font>

Es una buena idea utilizar un conjunto de validación. Esta es una muestra de los datos que retenemos de nuestro análisis y modelado. Lo usamos justo al final de nuestro proyecto para confirmar el Accuracy de nuestro modelo final. Es una prueba que podemos usar para ver si nos equivocamos y para darnos confianza en nuestras estimaciones de Accuracy en datos no etiquetados. Dividiremos el conjunto de datos cargado en dos, el 80% de los cuales utilizaremos para entrenar nuestros modelos y el 20% como un conjunto de datos de validación

In [29]:
# Split-out validation dataset
array = dataset.values
X = ???
Y = ???
validation_size = ???
seed = 7
X_train, X_validation, Y_train, Y_validation = ???

Ahora tiene datos de entrenamiento en `X_train` e `Y_train` para preparar modelos y conjuntos de `X_validation` e `Y_validation` que podemos usar más adelante.

<a id="section42"></a>
## <font color="#004D7F"> 4.2. Evaluación de línea base</font>

No sabemos qué algoritmos funcionarán bien en este conjunto de datos. Podemos intuir que algoritmos basados en la distancia como $k$-NN y SVM pueden funcionar bien. Utilizaremos la validación cruzada 10 veces. El conjunto de datos no es demasiado pequeño y esta es una buena configuración estándar. Evaluaremos algoritmos utilizando la métrica Accuracy ya que las clases estaban balanceadas. Esta es una métrica general que dará una idea rápida de cuán correcto es un modelo dado. Más útil en problemas de clasificación binaria como este.

Posteriormente, creamos una línea base de rendimiento en este problema y verifiquemos varios algoritmos diferentes. Seleccionaremos un conjunto de algoritmos diferentes capaces de trabajar en este problema de clasificación. Los seis algoritmos seleccionados incluyen:
* __Algoritmos lineales:__ LoR y LDA.
* __Algoritmos no lineales:__ CART, SVM, NB y $k$-NN.

In [30]:
# Spot Check Algorithms
models = []
models.append((???))
models.append((???))
models.append((???))
models.append((???))
models.append((???))
models.append((???))

Todos los algoritmos usan hiperparámetros predeterminados. Comparemos los algoritmos. Mostraremos la media y la desviación estándar de Accuracy para cada algoritmo a medida que lo calculemos y recopilemos los resultados para su uso posterior.

In [ ]:
# Test options and evaluation metric
num_folds = ???
seed = 7
scoring = ???

results = []
names = []
for name, model in models:
    kfold = ???
    cv_results = ???
    results.append(cv_results)
    names.append(name)
    print(f"{name}: {cv_results.mean()*100.0:,.2f}% ({cv_results.std()*100.0:,.2f}%)")

Estos son solo valores medios de Accuracy. Siempre es aconsejable observar la distribución de los valores de Accuracy calculados en los _folds_ de validación cruzada. Podemos hacerlo gráficamente usando Boxplot.

In [ ]:
# Compare Algorithms
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

Los resultados muestran una distribución estrecha de $k$-NN y SVM que es alentadora, lo que sugiere una baja variación. NB, además de tener un resultado malo tiene un varianza muy grande por lo que lo hace un algoritmo muy inconsisten para neustros datos. Sin embargo, $k$-NN tiene un comportamiento en cada _fold_ muy regular, lo que hace que sea un algoritmo óptimo.

Es posible que la distribución variada de los atributos tenga un efecto en el Accuracy de algoritmos como SVM. En la siguiente sección, repetiremos esta comprobación realizando una estandarización del conjunto de datos de entrenamiento.

<a id="section43"></a>
## <font color="#004D7F"> 4.3. Evaluar modelos: estandarización</font>

Sospechamos que las diferentes distribuciones de los datos en bruto pueden estar afectando negativamente a la habilidad de algunos de los algoritmos. Vamos a evaluar los mismos algoritmos con una copia estandarizada del conjunto de datos. Aquí es donde los datos se transforman de manera que cada atributo tenga un valor medio de cero y una desviación estándar de uno. 

También debemos evitar la fuga de datos cuando transformamos los datos. Una buena manera de evitar fugas es usar Pipelines que estandaricen los datos y construyan el modelo para cada _fold_ de la validación cruzada. De esa forma podemos obtener una estimación justa de cómo cada modelo con datos estandarizados podría funcionar en datos no vistos.

In [ ]:
# Standardize the dataset
pipelines = []
pipelines.append(('ScaledLoR', Pipeline([('Scaler', StandardScaler()),('LR', LogisticRegression())])))
???
???
???
???
???
results = []
names = []
for name, model in pipelines:
    kfold = KFold(n_splits=num_folds, random_state=seed, shuffle=True)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    print(f"{name}: {cv_results.mean()*100.0:,.2f}% ({cv_results.std()*100.0:,.2f}%)")

Ejecutar el ejemplo proporciona los resultados enumerados a continuación. Podemos ver que $k$-NN todavía está bien, incluso mejor que antes. También podemos ver que la estandarización de los datos ha elevado la habilidad de SVM para ser el algoritmo con mejor desempeño (aunque con más varianza que $k$-NN) probado hasta ahora.

In [ ]:
# Compare Algorithms
fig = plt.figure()
fig.suptitle('Scaled Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

Los resultados sugieren profundizar en los algoritmos SVM y $k$-NN. Es muy probable que una configuración más allá de la predeterminada pueda generar modelos con mejor desempeño.

<div style="text-align: right"> <font size=5>
    <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a>
</font></div>

---

<a id="section5"></a>
# <font color="#004D7F"> 5. Fase de Optimización</font>

En esta sección investigamos el ajuste de los parámetros para dos algoritmos que muestran mejores resultados, estos son: $k$-NN y SVM.

<a id="section51"></a>
## <font color="#004D7F"> 5.1. Optimización de _k_-NN</font>

Podemos comenzar ajustando el número de vecinos para $k$-NN. El número predeterminado de vecinos es 7. A continuación, probamos todos los valores impares de _k_ del 1 al 21, cubriendo el valor predeterminado de 7. Cada valor de _k_ se evalúa utilizando una validación cruzada 10 veces en el conjunto de datos estandarizado de entrenamiento. Podemos imprimir la configuración que resultó en el más alto Accuracy, así como la Accuracy de todos los valores probados. Ejecutando el ejemplo, vemos los resultados a continuación.

In [ ]:
# Tune scaled KNN
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
neighbors = [1,3,5,7,9,11,13,15,17,19,21]
param_grid = dict(n_neighbors=neighbors)
model = KNeighborsClassifier()
kfold = KFold(n_splits=num_folds, random_state=seed, shuffle=True)

grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)

grid_result = grid.fit(rescaledX, Y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"{param}: {mean*100.0:,.2f}% ({stdev*100.0:,.2f}%)")

Podemos ver que la configuración óptima es $k=1$. Esto es interesante ya que el algoritmo hará predicciones utilizando la instancia más similar en el conjunto de datos de entrenamiento solo.

<a id="section52"></a>
## <font color="#004D7F"> 5.2. Optimización de SVM</font>

Podemos ajustar dos parámetros clave del algoritmo SVM, el valor de _C_ (cuánto relajar el margen) y el tipo de _kernel_. El valor predeterminado para SVM (la clase SVC) es usar el _kernel_ de la función de base radial _(RBF)_ con un valor de _C_ establecido en 1.0. Al igual que con $k$-NN, realizaremos un `GridSearchCV` utilizando una validación cruzada de 10 con una copia estandarizada del conjunto de datos de entrenamiento. Intentaremos una serie de tipos de _kernel_ más simples y valores de _C_ con menos sesgo y más sesgo (menor que y más de 1.0 respectivamente). 

Ejecutar el ejemplo imprime la mejor configuración, la precisión y las precisiones para todas las combinaciones de configuración.

In [ ]:
# Tune scaled SVM
scaler = ???
rescaledX = ???
c_values = ???
kernel_values = ???
param_grid = ???
model = ???
kfold = KFold(n_splits=num_folds, random_state=seed, shuffle=True)
grid = ???
grid_result = grid.fit(rescaledX, Y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"{param}: {mean*100.0:,.2f}% ({stdev*100.0:,.2f}%)")

Podemos ver que la configuración más precisa fue SVM con un _kernel RBF_ y un valor _C_ de 2.0. La precisión del 86.7470% es aparentemente mejor de lo que $k$-NN podría lograr.

<div style="text-align: right"> <font size=5>
    <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a>
</font></div>

---

<a id="section6"></a>
# <font color="#004D7F"> 6. Algoritmos ensamblados</font>

Otra forma en que podemos mejorar el rendimiento de los algoritmos en este problema es mediante el uso de modelos ensamblados. En esta sección evaluaremos cuatro algoritmos ensamblados diferentes, dos tipo Boosting y dos tipo  Bagging:
* Métodos Boosting: AdaBoost (AB) y Gradient Boosting (GBM).
* Métodos Bagging: Random Forest (RF) y Extra Trees (ET).

Utilizaresmo una validación cruzada de 10. No se utiliza la estandarización de datos en este caso porque los cuatro algoritmos de conjunto se basan en árboles de decisión que son menos sensibles a las distribuciones de datos.

In [ ]:
# ensembles
ensembles = []
ensembles.append(('AB', AdaBoostClassifier()))
ensembles.append((???))
ensembles.append((???))
ensembles.append((???))
results = []
names = []
for name, model in ensembles:
    kfold = KFold(n_splits=num_folds, random_state=seed, shuffle=True)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    print(f"{name}: {mean*100.0:,.2f}% ({stdev*100.0:,.2f}%)")

Podemos ver que ambas técnicas proporcionan puntajes de Accuracy bajos (por debajo de 80%) con configuraciones predeterminadas. Podemos trazar la distribución de puntajes de Accruacy en los _folds_ de validación cruzada.

In [ ]:
# Compare Algorithms
fig = plt.figure()
fig.suptitle('Ensemble Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

Los resultados sugieren que GBM puede ser digno de un estudio adicional, con una media fuerte y una extensión que se inclina hacia el 90% de Accuracy.

<div style="text-align: right"> <font size=5>
    <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a>
</font></div>

---

<a id="section7"></a>
# <font color="#004D7F"> 7. Fase de Forecasting</font>

El SVM mostró el mejor resultado como un modelo estable y de baja complejidad para este problema. En esta sección finalizaremos el modelo entrenándolo en todo el conjunto de datos de entrenamiento y haremos predicciones para el conjunto de datos de validación para confirmar nuestros hallazgos. 

Una parte de los resultados fue que SVM funciona mejor cuando el conjunto de datos está estandarizado, de modo que todos los atributos tienen un valor medio de cero y una desviación estándar de uno. Podemos calcular esto a partir del conjunto de datos de entrenamiento completo y aplicar la misma transformación a los atributos de entrada del conjunto de datos de validación.

In [ ]:
# Finalize Model
# prepare the model
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
model = SVC(C=1.5)
model.fit(rescaledX, Y_train)
# estimate accuracy on validation dataset
rescaledValidationX = scaler.transform(X_validation)
predictions = model.predict(rescaledValidationX)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

Podemos ver que alcanzamos una precisión de casi el 86% en el conjunto de datos de validación extendido. Una puntuación que coincide estrechamente con nuestras expectativas estimadas anteriormente durante el ajuste de SVM.

<div style="text-align: right"> <font size=5>
    <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a>
</font></div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>